# Module 4: M/M/1 Model Validation on Kubernetes
## Theoretical Predictions vs Real Cloud-Native System Measurements

**Learning Objectives:**
- Estimate service rate from real K8s system measurements using Istio metrics
- Validate M/M/1 theoretical formulas against measured cloud-native metrics
- Analyze model accuracy under varying load conditions on Kubernetes
- Understand practical deviations from theoretical predictions in service mesh environment
- Compare single vs multi-replica scaling behavior

---

## Methodology

### Service Rate Estimation Strategy for K8s

Instead of estimating service rate from individual response times, we use cloud-native metrics:

1. **Calibration Campaign**: Run controlled experiment with known load on K8s
2. **Collect Istio Metrics**: Measure utilization and throughput via Prometheus + Istio telemetry
3. **Calculate Service Rate**: μ = throughput / utilization (using native K8s CPU metrics)
4. **Validation Experiments**: Test multiple load conditions using estimated μ
5. **Scaling Analysis**: Compare single vs multi-replica performance

This approach leverages cloud-native observability to provide robust service rate estimation.

## Prerequisites and Setup

This notebook requires a Kubernetes cluster with Istio service mesh and Prometheus monitoring. Follow these steps to set up your environment:

### Required Components

1. **Kubernetes Cluster**
   - Minikube, kind, or any K8s cluster with sufficient resources (4+ GB RAM recommended)
   - `kubectl` configured and connected to your cluster

2. **Helm Package Manager**
   ```bash
   # Install Helm (if not already installed)
   curl https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3 | bash
   ```

3. **Istio Service Mesh**
   ```bash
   # Download and install Istio
   curl -L https://istio.io/downloadIstio | sh -
   export PATH="$PWD/istio-*/bin:$PATH"
   
   # Install Istio with default configuration
   istioctl install --set values.defaultRevision=default -y
   
   # Enable sidecar injection for our namespace
   kubectl create namespace spe-system
   kubectl label namespace spe-system istio-injection=enabled
   ```

4. **kube-prometheus-stack (Prometheus + Grafana)**
   ```bash
   # Add Prometheus community Helm repository
   helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
   helm repo update
   
   # Install kube-prometheus-stack
   helm install prometheus prometheus-community/kube-prometheus-stack \
     --namespace monitoring \
     --create-namespace \
     --set prometheus.prometheusSpec.serviceMonitorSelectorNilUsesHelmValues=false \
     --set prometheus.prometheusSpec.podMonitorSelectorNilUsesHelmValues=false
   ```

### Quick Setup Script

For a rapid local setup using minikube:

```bash
# Start minikube with sufficient resources
minikube start --memory=6144 --cpus=4

# Install all components
./setup-k8s-environment.sh
```

### Verification

After installation, verify all components are running:

```bash
# Check Istio
kubectl get pods -n istio-system

# Check Prometheus stack
kubectl get pods -n monitoring

# Check if ServiceMonitor CRDs are available
kubectl get crd servicemonitors.monitoring.coreos.com
```

---

**⚠️ Important Notes:**
- This setup may take 10-15 minutes depending on your internet connection
- Ensure your K8s cluster has sufficient resources (minimum 4GB RAM, 2 CPU cores)
- The notebook will automatically handle port-forwarding and service discovery

## Environment Verification

Verify that all required components are properly installed and configured before proceeding with experiments.

In [1]:
# =============================================================================
# IMPORTS AND DEPENDENCIES
# =============================================================================

import time
import numpy as np
import asyncio
import subprocess
import shutil
from typing import Dict, List, Optional, Tuple

# Import SPE modules
from k8s_utils import K8sManager, get_istio_queries
from mm1_theoretical import MM1Theoretical
from k8s_prometheus_collector import K8sPrometheusCollector  # NEW WORKING COLLECTOR!
from workload_generator import AsyncWorkloadGenerator

print("✅ All dependencies imported successfully")
print("📋 Available modules:")
print("  - K8sManager: Kubernetes deployment and management")
print("  - MM1Theoretical: M/M/1 theoretical calculations")  
print("  - K8sPrometheusCollector: NEW working K8s metrics collector")
print("  - AsyncWorkloadGenerator: Load testing and workload generation")

✅ All dependencies imported successfully
📋 Available modules:
  - K8sManager: Kubernetes deployment and management
  - MM1Theoretical: M/M/1 theoretical calculations
  - K8sPrometheusCollector: NEW working K8s metrics collector
  - AsyncWorkloadGenerator: Load testing and workload generation


In [ ]:
# =============================================================================
# K8S EXPERIMENT CONFIGURATION PARAMETERS
# =============================================================================

# K8s infrastructure
K8S_NAMESPACE = "spe-system"
K8S_MANIFESTS_DIR = "k8s-manifests"

# kube-prometheus-stack configuration
PROMETHEUS_NAMESPACE = "monitoring"
PROMETHEUS_SERVICE = "prometheus-kube-prometheus-prometheus"
PROMETHEUS_URL = "http://localhost:9090"

# Calibration experiment parameters
CALIBRATION_LAMBDA = 3.0      # Conservative arrival rate for calibration
CALIBRATION_DURATION = 300.0  # Duration of calibration experiment (seconds)
CALIBRATION_WARMUP = 60.0     # Warmup period to reach steady state

# Validation experiment parameters  
TARGET_UTILIZATIONS = [0.2, 0.4, 0.7]  # Multiple utilization levels
VALIDATION_DURATION = 180     # Duration per validation experiment (seconds)
VALIDATION_WARMUP = 60        # Warmup period per validation experiment
VALIDATION_COOLDOWN = 120      # Cooldown period between experiments

# Scaling experiment parameters
REPLICA_PLAN = [1, 2, 3]      # Pod replica counts to test
SCALING_TEST_LAMBDA = 8.0     # Fixed arrival rate for scaling tests
SCALING_DURATION = 120        # Duration per scaling test
SCALING_SETTLE_TIME = 30      # Wait time after scaling

# Metrics collection parameters
METRICS_STEP = "5s"           # Prometheus query resolution
CORE_METRICS = ['throughput', 'cpu_usage', 'response_time_avg']

print("K8s Experiment Configuration (kube-prometheus-stack):")
print(f"  Prometheus: {PROMETHEUS_SERVICE} in {PROMETHEUS_NAMESPACE} namespace")
print(f"  Calibration: λ={CALIBRATION_LAMBDA} req/s, duration={CALIBRATION_DURATION}s")
print(f"  Validation: {len(TARGET_UTILIZATIONS)} utilization levels, {VALIDATION_DURATION}s each")
print(f"  Scaling: {len(REPLICA_PLAN)} replica configurations, {SCALING_DURATION}s each")
total_time = (CALIBRATION_DURATION + len(TARGET_UTILIZATIONS) * VALIDATION_DURATION + len(REPLICA_PLAN) * SCALING_DURATION) / 60
print(f"  Total estimated time: {total_time:.1f} minutes")
print("Configuration complete")

K8s Experiment Configuration (kube-prometheus-stack):
  Prometheus: prometheus-kube-prometheus-prometheus in monitoring namespace
  Calibration: λ=3.0 req/s, duration=300.0s
  Validation: 4 utilization levels, 180s each
  Scaling: 3 replica configurations, 120s each
  Total estimated time: 23.0 minutes
Configuration complete


In [3]:
# =============================================================================
# K8S INFRASTRUCTURE DEPLOYMENT
# =============================================================================

def setup_k8s_infrastructure():
    """
    Deploy and configure K8s infrastructure for M/M/1 experiments.
    Uses kube-prometheus-stack for monitoring.
    
    Returns:
        Tuple of (k8s_manager, mm1_url, prometheus_ready)
    """
    print("K8s Infrastructure Setup (kube-prometheus-stack)")
    print("=" * 50)
    
    # Check if environment verification was run
    try:
        if not environment_status.get('prometheus', False):
            print("⚠️ Warning: kube-prometheus-stack might not be available")
            print("📋 Environment status:", environment_status)
            print("🔄 Continuing anyway, but monitoring may not work properly...")
            print("💡 Make sure to run the environment verification cell first\n")
    except NameError:
        print("⚠️ Warning: Environment verification not run")
        print("💡 Please run the environment verification cell first for best results")
        print("🔄 Continuing with infrastructure setup anyway...\n")
    
    # Check kubectl availability FIRST
    try:
        result = subprocess.run(['kubectl', 'version', '--client'], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode != 0:
            raise Exception("kubectl not available - cannot proceed")
        print("✅ kubectl available")
    except Exception as e:
        raise Exception(f"kubectl not available: {e}")
    
    # Setup namespace with proper configuration BEFORE any deployment
    print(f"\n🏗️ Creating namespace {K8S_NAMESPACE} FIRST...")
    try:
        # Create namespace 
        ns_create_result = subprocess.run([
            'kubectl', 'create', 'namespace', K8S_NAMESPACE
        ], capture_output=True, text=True, timeout=30)
        
        if ns_create_result.returncode == 0:
            print(f"✅ Namespace {K8S_NAMESPACE} created")
        elif "already exists" in ns_create_result.stderr:
            print(f"✅ Namespace {K8S_NAMESPACE} already exists")
        else:
            print(f"⚠️ Namespace creation issue: {ns_create_result.stderr}")
            print("🔄 Continuing anyway...")
        
        # Enable Istio injection IMMEDIATELY after namespace creation
        print(f"🔧 Enabling Istio injection on {K8S_NAMESPACE}...")
        label_result = subprocess.run([
            'kubectl', 'label', 'namespace', K8S_NAMESPACE, 
            'istio-injection=enabled', '--overwrite'
        ], capture_output=True, text=True, timeout=30)
        
        if label_result.returncode == 0:
            print(f"✅ Istio injection enabled on {K8S_NAMESPACE}")
        else:
            print(f"⚠️ Istio labeling issue: {label_result.stderr}")
            print("🔄 Continuing anyway...")
            
    except Exception as e:
        print(f"⚠️ Namespace setup error: {e}")
        print("🔄 Attempting to continue with deployment...")
    
    # Verify namespace exists before proceeding
    print(f"\n🔍 Verifying namespace {K8S_NAMESPACE} exists...")
    try:
        verify_result = subprocess.run([
            'kubectl', 'get', 'namespace', K8S_NAMESPACE
        ], capture_output=True, text=True, timeout=30)
        
        if verify_result.returncode == 0:
            print(f"✅ Namespace {K8S_NAMESPACE} verified and ready")
        else:
            raise Exception(f"Namespace {K8S_NAMESPACE} does not exist: {verify_result.stderr}")
    except Exception as e:
        raise Exception(f"Cannot proceed without namespace: {e}")
    
    # Initialize K8s manager AFTER namespace setup
    k8s = K8sManager(namespace=K8S_NAMESPACE)
    
    # Clean up any existing problematic ServiceMonitors
    print(f"\n🧹 Cleaning up old ServiceMonitors...")
    try:
        cleanup_result = subprocess.run([
            'kubectl', 'delete', 'servicemonitor', 'mm1-server-metrics', 
            '-n', K8S_NAMESPACE, '--ignore-not-found=true'
        ], capture_output=True, text=True, timeout=30)
        
        if cleanup_result.returncode == 0:
            print("✅ Cleaned up old ServiceMonitors")
        else:
            print(f"⚠️ ServiceMonitor cleanup warning: {cleanup_result.stderr}")
    except Exception as e:
        print(f"⚠️ ServiceMonitor cleanup warning: {e}")
    
    # NOW deploy manifests (namespace exists)
    print(f"\n🚀 Deploying manifests from {K8S_MANIFESTS_DIR}/...")
    if not k8s.deploy_manifests(K8S_MANIFESTS_DIR):
        raise Exception("K8s deployment failed")
    print("✅ Manifests deployed successfully")
    
    # Verify ServiceMonitor configuration
    print(f"\n📊 Verifying monitoring configuration...")
    try:
        # Check ServiceMonitors
        sm_result = subprocess.run([
            'kubectl', 'get', 'servicemonitor', '-n', K8S_NAMESPACE
        ], capture_output=True, text=True, timeout=30)
        
        if sm_result.returncode == 0:
            print("✅ ServiceMonitors configured:")
            for line in sm_result.stdout.strip().split('\n')[1:]:  # Skip header
                if line.strip():
                    sm_name = line.split()[0]
                    print(f"  - {sm_name}")
        else:
            print(f"⚠️ ServiceMonitor check warning: {sm_result.stderr}")
    except Exception as e:
        print(f"⚠️ ServiceMonitor verification warning: {e}")
    
    # Check pod readiness
    print("\n🔍 Checking pod status...")
    pod_status = k8s.get_pod_status()
    if 'error' in pod_status:
        raise Exception(f"Pod status check failed: {pod_status['error']}")
    
    print(f"📊 Pod status: {pod_status['ready_pods']}/{pod_status['total_pods']} ready")
    if pod_status['ready_pods'] == 0:
        raise Exception("No ready pods found")
    
    # Setup networking
    print("\n🌐 Setting up networking...")
    gateway_ip = k8s.get_gateway_ip()
    
    if gateway_ip and gateway_ip != "localhost":
        mm1_url = f"http://{gateway_ip}"
        print(f"🎯 Using external gateway: {mm1_url}")
    else:
        # Setup gateway port-forward
        print("🔗 Setting up gateway port-forward...")
        gateway_ready = k8s.setup_port_forward(
            service="istio-ingressgateway",
            local_port=8080,
            service_port=80,
            namespace="istio-system"
        )
        if not gateway_ready:
            raise Exception("Gateway port-forward failed")
        
        mm1_url = "http://localhost:8080"
        print(f"🎯 Using port-forward: {mm1_url}")
    
    # Setup Prometheus port-forward (kube-prometheus-stack)
    print(f"\n📊 Setting up Prometheus port-forward (kube-prometheus-stack)...")
    prometheus_ready = k8s.setup_port_forward(
        service=PROMETHEUS_SERVICE,
        local_port=9090,
        service_port=9090,
        namespace=PROMETHEUS_NAMESPACE
    )
    
    if not prometheus_ready:
        print("⚠️ Prometheus port-forward failed - trying alternative service names...")
        # Try alternative service names for kube-prometheus-stack
        alternative_services = [
            "prometheus-prometheus-kube-prometheus-prometheus",
            "kube-prometheus-stack-prometheus",
            "prometheus-server"
        ]
        
        for alt_service in alternative_services:
            print(f"🔗 Trying service name: {alt_service}")
            prometheus_ready = k8s.setup_port_forward(
                service=alt_service,
                local_port=9090,
                service_port=9090,
                namespace=PROMETHEUS_NAMESPACE
            )
            if prometheus_ready:
                print(f"✅ Connected using service: {alt_service}")
                break
    
    # Test connectivity
    print("\n🔍 Testing service connectivity...")
    success, response_time, message = k8s.test_service_connectivity(mm1_url)
    if not success:
        raise Exception(f"Service connectivity test failed: {message}")
    
    print(f"✅ Service reachable: {response_time:.3f}s ({message})")
    
    # Test Prometheus connectivity
    if prometheus_ready:
        print("🔍 Testing Prometheus connectivity...")
        try:
            prometheus_success, prometheus_time, prometheus_message = k8s.test_service_connectivity(PROMETHEUS_URL + "/api/v1/query?query=up")
            if prometheus_success:
                print(f"✅ Prometheus reachable: {prometheus_time:.3f}s")
            else:
                print(f"⚠️ Prometheus connectivity issue: {prometheus_message}")
        except Exception as e:
            print(f"⚠️ Prometheus test failed: {e}")
    else:
        print("❌ Could not establish Prometheus port-forward")
        print("💡 Check if kube-prometheus-stack is properly installed:")
        print("   kubectl get svc -n monitoring | grep prometheus")
    
    print("\n🎉 K8s infrastructure setup complete!")
    print(f"📋 Summary:")
    print(f"  - Namespace: {K8S_NAMESPACE} (with Istio injection)")
    print(f"  - M/M/1 Service: {mm1_url}")
    print(f"  - Prometheus: {PROMETHEUS_URL} {'(connected)' if prometheus_ready else '(connection issues)'}")
    print(f"  - Monitoring: kube-prometheus-stack in {PROMETHEUS_NAMESPACE} namespace")
    print(f"  - Metrics: Istio proxy metrics only (no application /metrics endpoint)")
    
    return k8s, mm1_url, prometheus_ready

# Execute infrastructure setup
k8s_manager, mm1_service_url, prometheus_available = setup_k8s_infrastructure()

K8s Infrastructure Setup (kube-prometheus-stack)
⚠️ Warning: Environment verification not run
💡 Please run the environment verification cell first for best results
🔄 Continuing with infrastructure setup anyway...

✅ kubectl available

🏗️ Creating namespace spe-system FIRST...
✅ Namespace spe-system already exists
🔧 Enabling Istio injection on spe-system...
✅ Istio injection enabled on spe-system

🔍 Verifying namespace spe-system exists...
✅ Namespace spe-system verified and ready

🧹 Cleaning up old ServiceMonitors...
✅ Cleaned up old ServiceMonitors

🚀 Deploying manifests from k8s-manifests/...
🚀 Deploying K8s manifests from k8s-manifests/...
✅ Manifests applied successfully
gateway.networking.istio.io/mm1-gateway unchanged
virtualservice.networking.istio.io/mm1-virtualservice unchanged
destinationrule.networking.istio.io/mm1-destinationrule unchanged
peerauthentication.security.istio.io/mm1-peer-auth unchanged
telemetry.telemetry.istio.io/mm1-telemetry unchanged
deployment.apps/mm1-se

## Service Rate Calibration Campaign

Run a controlled experiment to estimate the service rate of our M/M/1 system using K8s and Istio metrics.

In [4]:
# =============================================================================
# K8S CALIBRATION CAMPAIGN EXECUTION
# =============================================================================

async def run_k8s_calibration_campaign():
    """
    Execute K8s service rate calibration campaign using the NEW working collector.
    
    Returns:
        estimated_mu: Estimated service rate
        estimation_stats: Calibration statistics
    """
    print("K8s Service Rate Calibration Campaign (NEW COLLECTOR)")
    print("=" * 52)
    print(f"Calibration parameters:")
    print(f"  Target arrival rate: {CALIBRATION_LAMBDA} req/s")
    print(f"  Experiment duration: {CALIBRATION_DURATION} seconds")
    print(f"  Warmup period: {CALIBRATION_WARMUP} seconds")
    print(f"  Measurement period: {CALIBRATION_DURATION - CALIBRATION_WARMUP} seconds")
    print(f"  Service endpoint: {mm1_service_url}")

    # System health check before calibration - USING NEW COLLECTOR
    collector = K8sPrometheusCollector(PROMETHEUS_URL)
    if not collector.health_check():
        collector.close()
        raise Exception("Prometheus not accessible - cannot proceed")

    print("\n✅ Prometheus connectivity confirmed with NEW collector")
    collector.close()

    print("\nStarting K8s calibration experiment...")
    print("This will take approximately 4 minutes")
    
    # Initialize generators and collectors - USING NEW COLLECTOR
    async_generator = AsyncWorkloadGenerator(target_url=mm1_service_url, timeout=300.0)
    collector = K8sPrometheusCollector(PROMETHEUS_URL)
    
    start_time = time.time()
    
    # Generate calibration workload
    print("\n🚀 Running calibration workload...")
    calibration_results = await async_generator.generate_workload(
        lambda_rate=CALIBRATION_LAMBDA,
        duration=CALIBRATION_DURATION,
        verbose=True
    )
    
    end_time = time.time()
    
    print("\n📊 Collecting K8s/Istio metrics with NEW collector...")
    
    # Collect metrics for steady-state period (skip warmup) - USING NEW METHOD
    steady_state_start = start_time + CALIBRATION_WARMUP
    calibration_metrics = collector.collect_k8s_metrics(
        start_time=steady_state_start,
        end_time=end_time,
        metrics=['throughput', 'cpu_usage'],  # Core metrics for calibration
        step="5s"
    )
    
    collector.close()
    
    print(f"\n✅ Calibration experiment completed:")
    print(f"  Workload requests: {calibration_results.total_requests}")
    print(f"  Workload rate: {calibration_results.actual_rate:.2f} req/s")
    print(f"  Success rate: {calibration_results.success_rate:.1%}")
    print(f"  Average response time: {calibration_results.average_response_time:.3f}s")
    
    # Process calibration data using K8s metrics
    print("\n🔍 Processing K8s/Istio calibration metrics...")
    
    # Extract CPU utilization from K8s metrics (already in 0-1 range)
    utilizations = []
    if 'cpu_usage' in calibration_metrics:
        cpu_data = calibration_metrics['cpu_usage']
        utilizations = [v for v in cpu_data.values if v > 0 and v <= 1.0]  # K8s CPU as fraction
        print(f"  Collected {len(utilizations)} CPU utilization measurements")
        if utilizations:
            print(f"  CPU range: {np.min(utilizations):.4f} - {np.max(utilizations):.4f}")
    else:
        print("  ⚠️ No K8s CPU usage data available")

    # Extract throughput from Istio metrics
    throughputs = []
    if 'throughput' in calibration_metrics:
        throughput_data = calibration_metrics['throughput']
        throughputs = [v for v in throughput_data.values if v > 0]
        print(f"  Collected {len(throughputs)} Istio throughput measurements")
        if throughputs:
            print(f"  Throughput range: {np.min(throughputs):.2f} - {np.max(throughputs):.2f} req/s")
    else:
        print("  ⚠️ No Istio throughput data, using workload generator rate")
        # Fallback to workload generator rate
        throughputs = [calibration_results.actual_rate] * len(utilizations)

    if utilizations and throughputs:
        print(f"  Total valid data points: {len(utilizations)} CPU, {len(throughputs)} throughput")
        
        # Align the arrays (take minimum length)
        min_length = min(len(utilizations), len(throughputs))
        aligned_utilizations = utilizations[:min_length]
        aligned_throughputs = throughputs[:min_length]
        
        # Estimate service rate using K8s-specific method
        estimated_mu, estimation_stats = MM1Theoretical.estimate_service_rate_from_k8s_metrics(
            aligned_utilizations, aligned_throughputs
        )
        
        if estimated_mu:
            print(f"\n📈 K8s Service Rate Estimation Results:")
            print(f"  Estimated μ (median): {estimated_mu:.2f} req/s")
            print(f"  Mean μ: {estimation_stats['mean_mu']:.2f} req/s")
            print(f"  Standard deviation: {estimation_stats['std_mu']:.2f}")
            print(f"  Coefficient of variation: {estimation_stats['cv_mu']:.3f}")
            print(f"  Range: {estimation_stats['min_mu']:.2f} - {estimation_stats['max_mu']:.2f} req/s")
            print(f"  Valid measurements: {estimation_stats['valid_measurements']}/{estimation_stats['total_measurements']}")
            
            # Implied service time
            estimated_service_time = 1.0 / estimated_mu
            print(f"  Implied service time: {estimated_service_time:.4f} seconds")
            
        else:
            print(f"\n❌ K8s service rate estimation failed:")
            print(f"  Error: {estimation_stats.get('error', 'Unknown error')}")
            estimated_mu = 5.0  # Conservative fallback for K8s
            estimation_stats = {'error': 'Using fallback value for K8s'}
            print(f"  Using fallback μ = {estimated_mu} req/s")
    else:
        print("\n⚠️ Insufficient K8s calibration data - using fallback service rate")
        estimated_mu = 5.0  # Conservative for K8s environment
        estimation_stats = {'error': 'Insufficient K8s data'}

    print(f"\n🎉 K8s calibration phase complete")
    print(f"Using μ = {estimated_mu:.2f} req/s for K8s validation experiments")
    
    return estimated_mu, estimation_stats

# Execute K8s calibration campaign
estimated_mu, k8s_calibration_stats = await run_k8s_calibration_campaign()

K8s Service Rate Calibration Campaign (NEW COLLECTOR)
Calibration parameters:
  Target arrival rate: 3.0 req/s
  Experiment duration: 300.0 seconds
  Warmup period: 60.0 seconds
  Measurement period: 240.0 seconds
  Service endpoint: http://34.31.117.144

✅ Prometheus connectivity confirmed with NEW collector

Starting K8s calibration experiment...
This will take approximately 4 minutes

🚀 Running calibration workload...
Starting ASYNC workload generation:
  Target rate: 3.0 req/s
  Duration: 300.0 seconds
  URL: http://34.31.117.144/
  Scheduled 918 requests
  Scheduled 50 requests, rate: 2.56 req/s
  Scheduled 100 requests, rate: 2.70 req/s
  Scheduled 150 requests, rate: 2.86 req/s
  Scheduled 200 requests, rate: 3.05 req/s
  Scheduled 250 requests, rate: 2.99 req/s
  Scheduled 300 requests, rate: 3.01 req/s
  Scheduled 350 requests, rate: 2.96 req/s
  Scheduled 400 requests, rate: 2.98 req/s
  Scheduled 450 requests, rate: 2.98 req/s
  Scheduled 500 requests, rate: 3.06 req/s
  Sch

## M/M/1 Validation Experiments

Test the estimated service rate by running experiments at different target utilization levels. Each experiment will:

**Experiment Design:**
- Target different utilization levels (ρ = 0.2, 0.4, 0.6, 0.8) using estimated μ
- Calculate required arrival rate: λ = target_utilization × estimated_μ 
- Run workload for each utilization level with proper cooldown periods
- Collect K8s CPU metrics and Istio throughput/response time metrics
- Compare measured results with M/M/1 theoretical predictions

**Validation Methodology:**
- **Theoretical**: Use MM1Theoretical.calculate_metrics(λ, μ) for predictions
- **Measured**: Collect real metrics using K8sPrometheusCollector
- **Comparison**: Statistical correlation and error analysis between theory and practice

**Expected Behavior:**
- Low utilization (ρ=0.2): Excellent agreement with theory
- Medium utilization (ρ=0.4-0.6): Good agreement with minor deviations  
- High utilization (ρ=0.8): Potential deviations due to system limitations

This validates whether our estimated μ accurately predicts system behavior under varying loads.

In [ ]:
# =============================================================================
# K8S M/M/1 VALIDATION EXPERIMENTS
# =============================================================================

async def run_k8s_validation_experiments():
    """
    Execute M/M/1 validation experiments at different utilization levels.
    
    Uses the estimated service rate from calibration to test theoretical predictions
    against measured K8s/Istio metrics.
    
    Returns:
        k8s_theoretical_predictions: Dict {lambda_rate: theoretical_metrics}
        k8s_measured_metrics: Dict {lambda_rate: measured_metrics}
    """
    print("K8s M/M/1 Validation Experiments")
    print("=" * 33)
    print(f"Using estimated μ = {estimated_mu:.2f} req/s from calibration")
    print(f"Testing {len(TARGET_UTILIZATIONS)} utilization levels: {TARGET_UTILIZATIONS}")
    print(f"Experiment duration: {VALIDATION_DURATION}s each")
    print(f"Warmup period: {VALIDATION_WARMUP}s each")
    print(f"Cooldown between experiments: {VALIDATION_COOLDOWN}s")
    
    total_time = len(TARGET_UTILIZATIONS) * (VALIDATION_DURATION + VALIDATION_COOLDOWN) / 60
    print(f"Estimated total time: {total_time:.1f} minutes")
    
    k8s_theoretical_predictions = {}
    k8s_measured_metrics = {}
    
    for i, target_utilization in enumerate(TARGET_UTILIZATIONS):
        # Calculate required arrival rate for target utilization
        target_lambda = target_utilization * estimated_mu
        
        print(f"\n" + "="*60)
        print(f"Validation Experiment {i+1}/{len(TARGET_UTILIZATIONS)}")
        print(f"Target utilization: ρ = {target_utilization:.1f}")
        print(f"Required arrival rate: λ = {target_lambda:.2f} req/s")
        print(f"Using service rate: μ = {estimated_mu:.2f} req/s")
        print("="*60)
        
        # Cooldown period (except for first experiment)
        if i > 0:
            print(f"\n⏳ Cooldown period: {VALIDATION_COOLDOWN}s")
            print("Allowing system to settle before next experiment...")
            time.sleep(VALIDATION_COOLDOWN)
        
        # Calculate theoretical predictions
        print(f"\n📊 Calculating theoretical M/M/1 predictions...")
        theoretical_metrics = MM1Theoretical.calculate_metrics(target_lambda, estimated_mu)
        k8s_theoretical_predictions[target_lambda] = theoretical_metrics
        
        if not theoretical_metrics['stable']:
            print(f"⚠️ WARNING: System unstable (λ ≥ μ), skipping experiment")
            continue
            
        print(f"✅ Theoretical predictions:")
        print(f"  Utilization: {theoretical_metrics['utilization']:.3f}")
        print(f"  Throughput: {theoretical_metrics['throughput']:.2f} req/s")
        print(f"  Response time: {theoretical_metrics['response_time']:.3f}s")
        print(f"  Queue length: {theoretical_metrics['queue_length']:.2f}")
        
        # Run validation experiment
        print(f"\n🚀 Running validation experiment...")
        print(f"Generating workload at λ = {target_lambda:.2f} req/s for {VALIDATION_DURATION}s")
        
        async_generator = AsyncWorkloadGenerator(target_url=mm1_service_url, timeout=300.0)
        collector = K8sPrometheusCollector(PROMETHEUS_URL)
        
        start_time = time.time()
        
        # Generate workload
        workload_results = await async_generator.generate_workload(
            lambda_rate=target_lambda,
            duration=VALIDATION_DURATION,
            verbose=False
        )
        
        end_time = time.time()
        
        print(f"✅ Workload completed:")
        print(f"  Target rate: {target_lambda:.2f} req/s")
        print(f"  Actual rate: {workload_results.actual_rate:.2f} req/s")
        print(f"  Success rate: {workload_results.success_rate:.1%}")
        print(f"  Total requests: {workload_results.total_requests}")
        print(f"  Avg response time: {workload_results.average_response_time:.3f}s")
        
        # Collect K8s/Istio metrics (skip warmup period)
        print(f"\n📊 Collecting K8s/Istio metrics...")
        steady_state_start = start_time + VALIDATION_WARMUP
        
        validation_metrics = collector.collect_k8s_metrics(
            start_time=steady_state_start,
            end_time=end_time,
            metrics=['throughput', 'cpu_usage'],  # Same as calibration - WORKING!
            step=METRICS_STEP
        )
        
        collector.close()
        
        # Process measured metrics
        print(f"📈 Processing measured metrics...")
        measured_metrics = {
            'lambda_rate': target_lambda,
            'utilization': None,
            'throughput': None,
            'response_time': None,
            'response_time_client': workload_results.average_response_time,
            'response_time_istio': None,
            'response_time_source': 'client-side',  # Default
            'success_rate': workload_results.success_rate,
            'total_requests': workload_results.total_requests,
            'actual_rate': workload_results.actual_rate
        }
        
        # Extract CPU utilization (K8s native metrics)
        if 'cpu_usage' in validation_metrics and validation_metrics['cpu_usage'].values:
            cpu_values = [v for v in validation_metrics['cpu_usage'].values if v > 0 and v <= 1.0]
            if cpu_values:
                measured_metrics['utilization'] = np.mean(cpu_values)
                print(f"  K8s CPU utilization: {measured_metrics['utilization']:.3f} ({len(cpu_values)} samples)")
            else:
                print(f"  ⚠️ No valid K8s CPU utilization data")
        else:
            print(f"  ⚠️ No K8s CPU usage metrics available")
        
        # Extract throughput (Istio metrics)
        if 'throughput' in validation_metrics and validation_metrics['throughput'].values:
            throughput_values = [v for v in validation_metrics['throughput'].values if v > 0]
            if throughput_values:
                measured_metrics['throughput'] = np.mean(throughput_values)
                print(f"  Istio throughput: {measured_metrics['throughput']:.2f} req/s ({len(throughput_values)} samples)")
            else:
                print(f"  ⚠️ No valid Istio throughput data, using workload rate")
                measured_metrics['throughput'] = workload_results.actual_rate
        else:
            print(f"  ⚠️ No Istio throughput metrics, using workload rate")
            measured_metrics['throughput'] = workload_results.actual_rate
        
        # For response time, since we're not collecting Istio response_time_avg, use client-side
        measured_metrics['response_time'] = workload_results.average_response_time
        print(f"  Response time (client-side): {measured_metrics['response_time']:.3f}s")
        
        # Store measured results
        k8s_measured_metrics[target_lambda] = measured_metrics
        
        # Compare with theoretical predictions
        print(f"\n📋 Experiment {i+1} Summary:")
        print(f"  Theoretical vs Measured:")
        if measured_metrics['utilization']:
            util_error = abs(theoretical_metrics['utilization'] - measured_metrics['utilization']) / theoretical_metrics['utilization'] * 100
            print(f"    Utilization: {theoretical_metrics['utilization']:.3f} vs {measured_metrics['utilization']:.3f} ({util_error:.1f}% error)")
        
        if measured_metrics['throughput']:
            tp_error = abs(theoretical_metrics['throughput'] - measured_metrics['throughput']) / theoretical_metrics['throughput'] * 100
            print(f"    Throughput: {theoretical_metrics['throughput']:.2f} vs {measured_metrics['throughput']:.2f} req/s ({tp_error:.1f}% error)")
        
        if measured_metrics['response_time']:
            rt_error = abs(theoretical_metrics['response_time'] - measured_metrics['response_time']) / theoretical_metrics['response_time'] * 100
            print(f"    Response time: {theoretical_metrics['response_time']:.3f} vs {measured_metrics['response_time']:.3f}s ({rt_error:.1f}% error)")
    
    print(f"\n🎉 K8s M/M/1 validation experiments completed!")
    print(f"✅ Completed {len(k8s_measured_metrics)} validation experiments")
    print(f"📊 Ready for validation analysis and plotting")
    
    return k8s_theoretical_predictions, k8s_measured_metrics

# Execute K8s validation experiments
k8s_theoretical_predictions, k8s_measured_metrics = await run_k8s_validation_experiments()

## Validation Results Analysis and Visualization

Analyze the M/M/1 validation results by comparing theoretical predictions with measured K8s/Istio metrics.

**Analysis Components:**
- **Correlation Analysis**: Statistical correlation between theoretical and measured values
- **Error Quantification**: Mean Absolute Relative Errors (MARE) for each metric type
- **Visual Comparison**: Side-by-side plots of theory vs measurements
- **Source Analysis**: Comparison of client-side vs Istio service mesh response times

**Key Metrics Compared:**
- **Throughput**: Theoretical λ vs Istio request rate measurements
- **Response Time**: Theoretical E[T] vs measured latency (client and/or Istio)
- **Utilization**: Theoretical ρ vs K8s CPU utilization metrics

**Expected Insights:**
- Model accuracy across different utilization levels
- Impact of cloud-native observability on measurement precision
- Identification of system behavior deviations from theoretical predictions
- Validation of service rate estimation methodology

In [ ]:
# =============================================================================
# K8S VALIDATION RESULTS ANALYSIS AND PLOTTING
# =============================================================================

# Import K8s-specific plotting functions
from k8s_validation_plots import (
    plot_k8s_validation_analysis, 
    calculate_k8s_validation_statistics,
    plot_k8s_experiment_timeline
)

def analyze_k8s_validation_results():
    """
    Comprehensive analysis of K8s M/M/1 validation results.
    
    Performs statistical analysis and creates visualization plots comparing
    theoretical M/M/1 predictions with measured K8s/Istio metrics.
    """
    print("K8s M/M/1 Validation Results Analysis")
    print("=" * 37)
    print(f"Analyzing {len(k8s_measured_metrics)} validation experiments")
    print(f"Estimated service rate: μ = {estimated_mu:.2f} req/s")
    print(f"Tested utilization levels: {TARGET_UTILIZATIONS}")
    
    # Validate that we have data to analyze
    if not k8s_theoretical_predictions or not k8s_measured_metrics:
        print("❌ No validation data available for analysis")
        return
    
    # 1. Generate comprehensive validation plots
    print(f"\n📊 Generating K8s validation plots...")
    try:
        plot_k8s_validation_analysis(
            k8s_theoretical_predictions, 
            k8s_measured_metrics, 
            estimated_mu
        )
    except Exception as e:
        print(f"⚠️ Plotting error: {e}")
    
    # 2. Calculate statistical validation metrics
    print(f"\n📈 Computing statistical validation metrics...")
    try:
        k8s_validation_stats = calculate_k8s_validation_statistics(
            k8s_theoretical_predictions, 
            k8s_measured_metrics
        )
        
        # Display detailed statistical analysis
        print(f"\n📋 K8s Statistical Validation Summary:")
        print(f"=" * 37)
        
        if 'error' not in k8s_validation_stats:
            print(f"Experiments analyzed: {k8s_validation_stats['experiments_count']}")
            
            # Correlation analysis
            print(f"\n🔗 Correlation Coefficients (K8s/Istio vs Theory):")
            correlation_metrics = ['throughput_corr', 'response_time_corr', 'utilization_corr']
            for metric in correlation_metrics:
                if metric in k8s_validation_stats:
                    corr = k8s_validation_stats[metric]
                    p_val = k8s_validation_stats.get(metric.replace('_corr', '_p_value'), 'N/A')
                    metric_name = metric.replace('_corr', '').replace('_', ' ').title()
                    print(f"  {metric_name}: r = {corr:.3f} (p = {p_val})")
            
            # Error analysis
            print(f"\n📏 Mean Absolute Relative Errors (MARE):")
            error_metrics = ['throughput_mae', 'response_time_mae', 'utilization_mae']
            for metric in error_metrics:
                if metric in k8s_validation_stats:
                    error = k8s_validation_stats[metric]
                    metric_name = metric.replace('_mae', '').replace('_', ' ').title()
                    print(f"  {metric_name}: {error:.1f}%")
            
            # Overall assessment
            print(f"\n🎯 Overall K8s Model Assessment:")
            print(f"  Assessment: {k8s_validation_stats['assessment']}")
            print(f"  Minimum correlation: {k8s_validation_stats['min_correlation']:.3f}")
            print(f"  Maximum error: {k8s_validation_stats['max_error']:.1f}%")
            
            # Response time source analysis
            if 'response_time_sources' in k8s_validation_stats:
                print(f"\n📡 Response Time Measurement Sources:")
                sources = k8s_validation_stats['response_time_sources']
                for source, count in sources.items():
                    print(f"  {source}: {count} experiments")
        else:
            print(f"❌ Statistical analysis failed: {k8s_validation_stats['error']}")
    
    except Exception as e:
        print(f"⚠️ Statistical analysis error: {e}")
    
    # 3. Generate experiment timeline plot
    print(f"\n📈 Generating experiment timeline...")
    try:
        plot_k8s_experiment_timeline(
            k8s_measured_metrics,
            title="K8s M/M/1 Validation Experiment Timeline"
        )
    except Exception as e:
        print(f"⚠️ Timeline plotting error: {e}")
    
    # 4. Detailed per-experiment comparison
    print(f"\n📊 Detailed Per-Experiment Analysis:")
    print("=" * 37)
    
    for lambda_rate in sorted(k8s_measured_metrics.keys()):
        theoretical = k8s_theoretical_predictions.get(lambda_rate, {})
        measured = k8s_measured_metrics[lambda_rate]
        
        target_util = lambda_rate / estimated_mu
        print(f"\n🔬 Experiment: λ = {lambda_rate:.2f} req/s (ρ = {target_util:.2f})")
        
        # Throughput comparison
        if theoretical.get('throughput') and measured.get('throughput'):
            th_tp = theoretical['throughput']
            ms_tp = measured['throughput']
            tp_error = abs(th_tp - ms_tp) / th_tp * 100
            print(f"  Throughput: {th_tp:.2f} → {ms_tp:.2f} req/s ({tp_error:.1f}% error)")
        
        # Response time comparison
        if theoretical.get('response_time') and measured.get('response_time'):
            th_rt = theoretical['response_time']
            ms_rt = measured['response_time']
            rt_error = abs(th_rt - ms_rt) / th_rt * 100
            rt_source = measured.get('response_time_source', 'unknown')
            print(f"  Response time: {th_rt:.3f} → {ms_rt:.3f}s ({rt_error:.1f}% error, {rt_source})")
        
        # Utilization comparison
        if theoretical.get('utilization') and measured.get('utilization'):
            th_util = theoretical['utilization']
            ms_util = measured['utilization']
            util_error = abs(th_util - ms_util) / th_util * 100
            print(f"  Utilization: {th_util:.3f} → {ms_util:.3f} ({util_error:.1f}% error)")
        
        # Success rate
        success_rate = measured.get('success_rate', 0)
        if success_rate < 0.99:
            print(f"  ⚠️ Success rate: {success_rate:.1%} (< 99%)")
    
    # 5. Key insights and recommendations
    print(f"\n💡 K8s Validation Insights:")
    print("=" * 25)
    
    # Analyze utilization levels performance
    high_util_issues = []
    excellent_agreement = []
    
    for lambda_rate in sorted(k8s_measured_metrics.keys()):
        target_util = lambda_rate / estimated_mu
        measured = k8s_measured_metrics[lambda_rate]
        theoretical = k8s_theoretical_predictions.get(lambda_rate, {})
        
        if measured.get('success_rate', 1.0) < 0.99:
            high_util_issues.append(f"ρ={target_util:.1f}")
        
        # Check if all metrics have low error
        low_error_count = 0
        for metric in ['throughput', 'response_time', 'utilization']:
            th_val = theoretical.get(metric)
            ms_val = measured.get(metric)
            if th_val and ms_val:
                error = abs(th_val - ms_val) / th_val * 100
                if error < 15:  # Less than 15% error
                    low_error_count += 1
        
        if low_error_count >= 2:
            excellent_agreement.append(f"ρ={target_util:.1f}")
    
    print(f"✅ Excellent model agreement: {', '.join(excellent_agreement) if excellent_agreement else 'None'}")
    print(f"⚠️ High utilization issues: {', '.join(high_util_issues) if high_util_issues else 'None'}")
    
    # Cloud-native benefits
    print(f"\n🚀 Cloud-Native Observability Benefits:")
    istio_measurements = sum(1 for m in k8s_measured_metrics.values() 
                           if m.get('response_time_source') == 'Istio service mesh')
    total_measurements = len(k8s_measured_metrics)
    
    if istio_measurements > 0:
        print(f"  Istio service mesh metrics: {istio_measurements}/{total_measurements} experiments")
        print(f"  Native K8s CPU monitoring: CPU utilization without application instrumentation")
        print(f"  Service mesh benefits: Server-side response time measurements")
    
    print(f"\n🎉 K8s M/M/1 validation analysis complete!")

# Execute comprehensive validation analysis
analyze_k8s_validation_results()